## X,Y 좌표로 자치구/행정동 추출하기

In [1]:
import geopandas as gpd
import pandas as pd

In [10]:
# 공간 데이터 csv로 변환하여 저장
seoul = gpd.read_file("/Users/Jenny/Desktop/seoulbigcam/Raw Data/2020/TL_TBVIATR_FCLTY_INFO_2020.shp")

#csv 파일 저장
csv_file = "/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 주요시설과 집객시설 공간데이터(2020).csv"
seoul.to_csv(csv_file, index=False)


## 생략가능 아래 코드만 실행하면 됩니다

In [20]:
# 교육시설/교통시설 제외하기

seoul = seoul[seoul['MLSFCDC'] != '교육시설']
seoul = seoul[seoul['MLSFCDC'] != '교통시설']
seoul = seoul.reset_index(drop=True)
seoul

,STDR_YM,VIAT_SE_CD,VIAT_CD,VIAT_ID,LCLASDC,MLSFCDC,SCLASDC,DCLASDC,BCLASDC,VIAT_NM,...,ADRES_NM,TELNO,X_VALUE,Y_VALUE,ADRES_SP,BLCK_CD,REGIST_DE,ERSR_DE,REGIST_DT,geometry
0,None,070301,0302,AA0000000002,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,None,None,214555.910095,550962.356812,None,156617,None,None,None,POINT (214555.910 550962.357)
1,None,070301,0302,AA0000000005,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,고덕그라시움상가,...,None,None,214481.776611,550819.438110,None,155064,None,None,None,POINT (214481.777 550819.438)
2,None,070301,0302,AA0000000006,산업,서비스산업,종합상품판매업,대형상가,일반종합상가,그라시움상가,...,None,None,214465.779480,551177.935486,None,156674,None,None,None,POINT (214465.779 551177.935)
3,None,060203,0200,AA0000000007,레저/관광/예술,문화/예술시설,박물관,공립박물관,None,송파BOOK책박물관,...,None,None,209236.807129,544386.432007,None,154275,None,None,None,POINT (209236.807 544386.432)
4,None,070301,0400,AA0000000011,산업,서비스산업,종합상품판매업,시장,미분류,"가락시장,헬리오시티버스정류장쉘터",...,None,None,209386.103088,543972.747620,None,154048,None,None,None,POINT (209386.103 543972.748)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12899,None,070301,0600,AA0000019455,산업,서비스산업,종합상품판매업,기타종합상품판매업,None,None,...,None,None,190670.408835,541421.664715,None,19928,None,None,None,POINT (190670.409 541421.665)
12900,None,030204,0000,AA0000019456,교육/보건,보건시설,한의원,None,None,None,...,None,None,191989.932373,539010.519529,None,19304,None,None,None,POINT (191989.932 539010.520)
12901,None,070301,0200,AA0000019459,산업,서비스산업,종합상품판매업,할인매장,미분류,None,...,None,None,191354.443045,541322.297120,None,20337,None,None,None,POINT (191354.443 541322.297)
12902,None,030201,0800,AA0000019460,교육/보건,보건시설,병/의원,피부과,None,None,...,None,None,188687.329239,547251.976724,None,11699,None,None,None,POINT (188687.329 547251.977)


In [21]:
# X,Y좌표 경도 위도로 변환하기

gdf = gpd.GeoDataFrame(seoul, geometry=gpd.points_from_xy(seoul.X_VALUE, seoul.Y_VALUE))

gdf.crs = 'epsg:5186'
gdf = gdf.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y

# 필요한 컬럼만 추출해서 저장하기
gdf = gdf[['MLSFCDC', 'VIAT_NM', 'x', 'y']]

print(gdf)

       MLSFCDC            VIAT_NM           x          y
0        서비스산업           고덕그라시움상가  127.164740  37.558074
1        서비스산업           고덕그라시움상가  127.163898  37.556787
2        서비스산업             그라시움상가  127.163724  37.560018
3      문화/예술시설         송파BOOK책박물관  127.104457  37.498893
4        서비스산업  가락시장,헬리오시티버스정류장쉘터  127.106140  37.495165
...        ...                ...         ...        ...
12899    서비스산업               None  126.894531  37.472180
12900     보건시설               None  126.909474  37.450467
12901    서비스산업               None  126.902265  37.471291
12902     보건시설               None  126.872023  37.524689
12903     보건시설               None  127.033093  37.527229

[12904 rows x 4 columns]


In [15]:
gdf = gdf[:30]

### 경도, 위도를 이용해 REST API로 행정동/자치구 추출하기

In [13]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [22]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)


# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in gdf.iterrows():
    get_location(row)
    
    # 200개씩 처리될 때 메시지 출력
    if i % 200 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
gdf['city'] = [location.get('city') for location in locations]
gdf['gu'] = [location.get('gu') for location in locations]
gdf['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2800 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보 불러오는 중...
3200 번째 경위도 좌표계에 대한 정보 불러오는 중...
3400 번째 경위도 좌표계에 대한 정보 불러오는 중...
3600 번째 경위도 좌표계에 대한 정보 불러오는 중...
3800 번째 경위도 좌표계에 대한 정보 불러오는 중...
4000 번째 경위도 좌표계에 대한 정보 불러오는 중...
4200 번째 경위도 좌표계에 대한 정보 불러오는 중...
4400 번째 경위도 좌표계에 대한 정보 불러오는 중...
4600 번째 경위도 좌표계에 대한 정보 불러오는 중...
4800 번째 경위도 좌표계에 대한 정보 불러오는 중...
5000 번째 경위도 좌표계에 대한 정보 불러오는 중...
5200 번째 경위도 좌표계에 대한 정보 불러오는 중...
5400 번째 경위도 좌표계에 대한 정보 불러오는 중...
5600 번째 경위도 좌표계에 대한 정보 불러오는 중...
5800 번째 경위도 좌표계에 대한 정보 불러오는 중...
6000 번째 경위도 좌표계에 

In [25]:
# csv로 저장하기
gdf.to_csv("/Users/Jenny/Desktop/seoulbigcam/Preprocessed Data/서울시 주요시설과 집객시설 공간데이터(2020)_2차.csv")

In [23]:
gdf

,MLSFCDC,VIAT_NM,x,y,city,gu,dong
0,서비스산업,고덕그라시움상가,127.164740,37.558074,서울특별시,강동구,고덕2동
1,서비스산업,고덕그라시움상가,127.163898,37.556787,서울특별시,강동구,고덕2동
2,서비스산업,그라시움상가,127.163724,37.560018,서울특별시,강동구,고덕2동
3,문화/예술시설,송파BOOK책박물관,127.104457,37.498893,서울특별시,송파구,가락1동
4,서비스산업,"가락시장,헬리오시티버스정류장쉘터",127.106140,37.495165,서울특별시,송파구,가락1동
...,...,...,...,...,...,...,...
12899,서비스산업,None,126.894531,37.472180,서울특별시,금천구,독산1동
12900,보건시설,None,126.909474,37.450467,서울특별시,금천구,시흥5동
12901,서비스산업,None,126.902265,37.471291,서울특별시,금천구,독산3동
12902,보건시설,None,126.872023,37.524689,서울특별시,양천구,목1동


In [51]:
gdf_new = gdf[['MLSFCDC', 'city', 'gu', 'dong']]
gdf_dong = gdf.groupby(['city', 'gu', 'dong'], as_index=False)['MLSFCDC'].count()

# 서울시 데이터만 확인
gdf_dong = gdf_dong[gdf_dong['city'] == '서울특별시']
gdf_dong

,city,gu,dong,MLSFCDC
29,서울특별시,강남구,개포1동,3
30,서울특별시,강남구,개포2동,9
31,서울특별시,강남구,개포3동,3
32,서울특별시,강남구,개포4동,20
33,서울특별시,강남구,논현1동,88
...,...,...,...,...
450,서울특별시,중랑구,상봉2동,61
451,서울특별시,중랑구,신내1동,14
452,서울특별시,중랑구,신내2동,6
453,서울특별시,중랑구,중화1동,10


In [52]:
# 행정동별 면적 데이터 로드

area = pd.read_csv("/Users/Jenny/Desktop/seoulbigcam/Raw Data/행정동별 면적.csv")


area = area.drop(area.index[0:3])     # 필요없는 데이터 정리
area = area.reset_index(drop=True)    # 인덱스 리셋
area = area.drop('동별(1)', axis=1)    # 필요없는 컬럼 삭제
area = area.rename(columns={'동별(2)': 'gu',
                            '동별(3)':'dong',
                            '2022':'area_km2'})     # 컬럼명 변경

area.head()

,gu,dong,area_km2
0,종로구,소계,23.91
1,종로구,사직동,1.23
2,종로구,삼청동,1.49
3,종로구,부암동,2.27
4,종로구,평창동,8.87


In [61]:
# 데이터 병합

gdf_mer = gdf_dong.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
gdf_mer

,city,gu,dong,MLSFCDC,area_km2
0,서울특별시,강남구,개포1동,3,1.27
1,서울특별시,강남구,개포2동,9,2.51
2,서울특별시,강남구,개포3동,3,1.24
3,서울특별시,강남구,개포4동,20,1.49
4,서울특별시,강남구,논현1동,88,1.25
...,...,...,...,...,...
421,서울특별시,중랑구,상봉2동,61,0.65
422,서울특별시,중랑구,신내1동,14,2.56
423,서울특별시,중랑구,신내2동,6,0.93
424,서울특별시,중랑구,중화1동,10,0.64


In [62]:
gdf_mer['area_km2'] = gdf_mer['area_km2'].astype(float)
gdf_mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 0 to 425
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   city      426 non-null    object 
 1   gu        426 non-null    object 
 2   dong      426 non-null    object 
 3   MLSFCDC   426 non-null    int64  
 4   area_km2  426 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 20.0+ KB


In [63]:
# 면적당 근린시설 수 (근린시설 밀집도) 컬럼 생성
gdf_mer['density_vital'] = gdf_mer['MLSFCDC'] / gdf_mer['area_km2']
gdf_mer

,city,gu,dong,MLSFCDC,area_km2,density_vital
0,서울특별시,강남구,개포1동,3,1.27,2.362205
1,서울특별시,강남구,개포2동,9,2.51,3.585657
2,서울특별시,강남구,개포3동,3,1.24,2.419355
3,서울특별시,강남구,개포4동,20,1.49,13.422819
4,서울특별시,강남구,논현1동,88,1.25,70.400000
...,...,...,...,...,...,...
421,서울특별시,중랑구,상봉2동,61,0.65,93.846154
422,서울특별시,중랑구,신내1동,14,2.56,5.468750
423,서울특별시,중랑구,신내2동,6,0.93,6.451613
424,서울특별시,중랑구,중화1동,10,0.64,15.625000
